# 구글플레이에 등록된 앱의 리뷰 추출

In [442]:
# 기본적으로 불러와야되는 셀레니윰 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#사람인척 하기 위한 옵션으로 사람이 아닌 셀레니움이면 사이트에서 막히기 때문에 사용
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/141.0.0.0")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
#웹브라우저에서 url열기
driver.get("https://play.google.com/store/apps/details?id=viva.republica.toss&hl=ko")

In [78]:
import time

In [102]:
driver.execute_script(f"window.scrollTo({0},{1400})")
time.sleep(2)

In [103]:
# 평점 및 리뷰 옆의 -> 버튼 클릭
button = driver.find_element(By.CSS_SELECTOR,'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')
button.click()

In [104]:
# 최신순으로 리뷰를 정리하기 위해서 버튼 클릭
driver.find_element(By.ID,"sortBy_1").click()

In [105]:
# 최신을 찾아서 클릭
driver.find_element(By.CSS_SELECTOR,'span[aria-label*="최신"]').click()

In [30]:
# window를 스크롤하면 리뷰가 아닌 배경 페이지가 스크롤 됨

In [40]:
# 리뷰가 담긴 창을 찾아서 Javascript로 1000px씩 아래로 스크롤
driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0,10000)")

In [444]:
#리뷰일
review_date = driver.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML')

In [50]:
# 별점
rating = float(driver.find_element(By.CSS_SELECTOR,'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

In [54]:
#사용자 리뷰
user_reviwe = driver.find_element(By.CSS_SELECTOR, ".h3YV2d").text

In [445]:
# 회사 응답
company_reply = driver.find_element(By.CSS_SELECTOR, ".ras4vb").text

In [81]:
from datetime import datetime,timedelta

In [72]:
# 추출한 리뷰일을 날짜형 데이터로 변경
# datetime.strptime(yyyy-mm-dd, '%Y-%m-%d') 날짜형 데이터 타입으로 변환
review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
review_date = datetime.strptime(review_date,'%Y-%m-%d')

In [73]:
review_date

datetime.datetime(2025, 10, 1, 0, 0)

In [82]:
today = datetime.today()
one_month_ago = today - timedelta(days=30)
one_month_ago.date()

datetime.date(2025, 9, 27)

In [86]:
if review_date.date() < one_month_ago.date():
    break

SyntaxError: 'break' outside loop (224360330.py, line 2)

# 토스 정보 합치기

In [154]:
# 웹 정보 불러올때까지 기달려주는 코드
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

웹브라우저에 표시되는 요소들(버튼,아이콘)은 클릭할 수 있게 되기까지 시간이 필요. <br>
요소가 클릭이 가능해질 때 까지 기다리도록 코드를 작성해 주어야 함<br>
```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
```
추가가 필요

In [153]:
import time
import pandas as pd
from selenium import webdriver
from datetime import datetime,timedelta
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [312]:
def to_date(review_date):
    review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
    review_date = datetime.strptime(review_date,'%Y-%m-%d')
    return review_date.date()

In [353]:
apps = dict (토스 = "viva.republica.toss&hl=ko", 국민 = "com.kbstar.kbbank&hl=ko",하나 = "com.kebhana.hanapush&hl=ko",
            뱅크셀러드 = "com.rainist.banksalad2&hl=ko0",핀다 = "kr.co.finda.finda&hl=ko")

In [468]:
def app_teview_extractor(apps):
    #사람인척 하기 위한 옵션으로 사람이 아닌 셀레니움이면 사이트에서 막히기 때문에 사용
    options = Options()
    options.add_experimental_option("detach", True)
    options.add_argument("start-maximized")
    options.add_argument("Chrome/141.0.0.0")
    options.add_argument("lang=ko_KR")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )
    #웹브라우저에서 url열기
    driver.get(f"https://play.google.com/store/apps/details?id={id}")

    # 요소가 실행 가능해질 때 까지 기다리기(기다리는 최대 시간)
    wait = WebDriverWait(driver,10)
    # 평점 및 리뷰 옆의 -> 버튼이 활성화 되도록 스크롤을 1400px 아래로 내림
    driver.execute_script(f"window.scrollTo({0},{1400})")
    time.sleep(2)

    # 평점 및 리뷰 옆의 -> 버튼 클릭
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')))
    button.click()

    # 최신순으로 리뷰를 정리하기 위해서 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.ID, "sortBy_1"))).click()
    time.sleep(2)

    # 최신을 찾아서 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))).click()
    time.sleep(2)
    #스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기
    # 첫페이지 마지막 리뷰의 날짜 구하기 오늘 날짜랑 비교하기
    #스크롤 전과 후의 리뷰 개수 비교하기
    today = datetime.today()
    today = today.date()
    end_date = today - timedelta(days=7)
    end_date_str = end_date.strftime('%Y년 %m월 %d일')
    to_date(end_date_str)
    end_px = 1000
    len_review_list = len(driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe"))
    last_review = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")[-1]
    last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML'))

    while last_review_date >= end_date:
        driver.execute_script(f"document.querySelector('.fysCi.Vk3ZVd').scrollTo(0,{end_px})")
        time.sleep(2)
        last_review = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")[-1]
        last_review_date = to_date(last_review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML'))

        end_px += 10000
        print(len( driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")))
        print(f"last_review_date:{last_review_date},end_date:{end_date}")
    # 리뷰 목록 가져오기 
    review_list = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")

    result = {}
    cols = ('리뷰일','앱 이름','별점','사용자 리뷰','회사 응답')
    r = 10000

    for review in review_list:
        #리뷰일
        review_date = review.find_element(By.CSS_SELECTOR,".bp9Aid").get_attribute('innerHTML')
        review_date = review_date.replace(" ","").replace("년","-").replace("월","-").replace("일","")
        # 별점
        rating = float(review.find_element(By.CSS_SELECTOR,'div[aria-label*="별표 5개 만점에"]').get_attribute('aria-label').split()[3].replace("개를",""))

        #사용자 리뷰
        user_reviwe = review.find_element(By.CSS_SELECTOR, ".h3YV2d").text

        # 회사 응답
        try:
            company_reply = review.find_element(By.CSS_SELECTOR, "ras4vb").text
        except:
            company_reply = "회사응답 없음"
            
            values = (review_date,app,rating,user_reviwe,company_reply)
            for key,value in zip(cols,values):
                result.setdefault(key,[]).append(value)
                
    df = pd.DataFrame(result)
    return df

In [461]:
for app,id in apps.items():
    df = app_teview_extractor(app)
    display(df)
    df.to_csv(f"./data/{app}_앱 리뷰 모음.csv",index = False , encoding = "utf-8-sig")

20
last_review_date:2025-10-25,end_date:2025-10-20
40
last_review_date:2025-10-24,end_date:2025-10-20
60
last_review_date:2025-10-23,end_date:2025-10-20
80
last_review_date:2025-10-22,end_date:2025-10-20
100
last_review_date:2025-10-21,end_date:2025-10-20
120
last_review_date:2025-10-21,end_date:2025-10-20
140
last_review_date:2025-10-20,end_date:2025-10-20
160
last_review_date:2025-10-19,end_date:2025-10-20


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-24,토스,5.0,토스로 대출 받기 정말 편리해요 감사합니다 ^^ 저는 아이폰으로 대출 받았어요. 쿠...,회사응답 없음
1,2025-10-23,토스,5.0,편하게 사용할수있어서 좋아서요 감사합니다,회사응답 없음
2,2025-10-22,토스,5.0,좋아요 만족합니다 강력추천합니다 감사합니다,회사응답 없음
3,2025-10-22,토스,5.0,토스는 대단하다.,회사응답 없음
4,2025-10-22,토스,5.0,평점계산방식,회사응답 없음
5,2025-10-21,토스,3.0,좋긴한데 카드 밑에 깨진것 같은 구멍은 왜 있고 일반 블랙은 어디갔나요? 구멍 때문...,회사응답 없음
6,2025-10-21,토스,2.0,홍민택 데려가주세요 카카오톡 돌려내주세요,회사응답 없음
7,2025-10-21,토스,5.0,믿고쓰는토스 입니다 1.일단 한눈에 쉽게 들어오게 만들었어요 어떠한 은행 및 금융어...,회사응답 없음
8,2025-10-21,토스,5.0,"진짜 은행계의 혁신이라고 생각들정도로 간편한 UI , 혜택, 파격적인 캐시백 이벤트...",회사응답 없음
9,2025-10-21,토스,5.0,영어공부가 재밌어요.,회사응답 없음


20
last_review_date:2025-10-23,end_date:2025-10-20
40
last_review_date:2025-10-22,end_date:2025-10-20
60
last_review_date:2025-10-19,end_date:2025-10-20


""


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-24,하나,1.0,예수금 빼려고 보니까 그냥은 또 안빠지고 계좌 밀어야 된데서 계좌 해지 찾아보니 밀...,회사응답 없음
1,2025-10-24,하나,1.0,앱이 왜 안열리져 다른통장에서 이체를해서 볼려고해도볼수가없네여 출근 교통비 써야되는...,회사응답 없음
2,2025-10-24,하나,1.0,갤럭시 s24 울트라 순정 os 씁니다. 얼마전부터 접속하려고 할 때마다 개조된 o...,회사응답 없음
3,2025-10-23,하나,1.0,로그인 인증이 안되어서 삭제후 재설치 했더니 아예 안되네요,회사응답 없음
4,2025-10-22,하나,4.0,"입출금알림 요즘 너무너무너무늦게와요 실시간 알림아니면 무슨 의미죠..,",회사응답 없음
5,2025-10-22,하나,1.0,one ui 8.0 업데이트 이후 정상적인 OS 에서만 사용가능하다는 알림창 뜨면서...,회사응답 없음
6,2025-10-19,하나,4.0,"입,출금 푸쉬알람이 한참 있다가 오네요... 주말이라그런가? 통장도 알림 다 켜져있는데",회사응답 없음
7,2025-10-18,하나,5.0,편리하고 좋아요~~~,회사응답 없음
8,2025-10-17,하나,1.0,최악이야,회사응답 없음
9,2025-10-17,하나,5.0,좋다,회사응답 없음


,리뷰일,앱 이름,별점,사용자 리뷰,회사 응답
0,2025-10-25,뱅크셀러드,1.0,"업데이트 후 메모•태그 입력하는데 오래걸리고, 자주 오류나서 불편합니다. 메모 기능...",회사응답 없음
1,2025-10-25,뱅크셀러드,5.0,빗썸도 가상자산 연결할수있게해주세요,회사응답 없음
2,2025-10-25,뱅크셀러드,1.0,업데이트후 계속 꺼짐 실행시키면 계속 꺼집니다 인터넷이 원활하지 않다는 메세지도 가...,회사응답 없음


""


In [475]:
driver.find_elements(By.CSS_SELECTOR,".RHo1pe")

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0xa3fe43+66515]
	GetHandleVerifier [0x0xa3fe84+66580]
	(No symbol) [0x0x82da6b]
	(No symbol) [0x0x86b5ab]
	(No symbol) [0x0x89b086]
	(No symbol) [0x0x89667c]
	(No symbol) [0x0x895bf3]
	(No symbol) [0x0x7fe72d]
	(No symbol) [0x0x7fecae]
	(No symbol) [0x0x7ff14d]
	GetHandleVerifier [0x0xc97353+2521315]
	GetHandleVerifier [0x0xc922d3+2500707]
	GetHandleVerifier [0x0xa67c94+229924]
	GetHandleVerifier [0x0xa581f8+165768]
	GetHandleVerifier [0x0xa5ecad+193085]
	(No symbol) [0x0x7fe3e3]
	(No symbol) [0x0x7fdb60]
	GetHandleVerifier [0x0xddc20f+3852191]
	BaseThreadInitThunk [0x0x75d15d49+25]
	RtlInitializeExceptionChain [0x0x7782d6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x7782d661+561]


In [ ]:
review_list = driver.find_elements(By.CSS_SELECTOR,"div.RHo1pe")
for review in review_list:
    print(driver.find_elementｓ(By.CSS_SELECTOR,"div.RHo1pe")[0].get_attribute('innerHTML'))